# ДОМАШКА
(Насколько я поняла, нужно найти самые типичные действия суда, то есть именно сочетания (суд) гл + что-то (т.е. например "мера пресечения" не подходит))

(у меня почему-то не работает from nltk.collocations import * :C)

In [22]:
import nltk
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

Итак, мы пытаемся найти биграммы действий суда, т.е. 2+3 столбцы. Добавим стоп-слов, чтобы можно было воспльзоваться nltk finderом и поставить на это фильтр

In [7]:
pre_words = open('court-V-N.csv').read().split('\n')
words = []
for pre_word in pre_words:
    words += [word.strip().lower() for word in pre_word.split(' ,')][:3] + ['stop']

In [10]:
finder = nltk.collocations.BigramCollocationFinder.from_words(words)
finder.apply_ngram_filter(lambda w1, w2: 'суд' in (w1, w2) or 'stop' in (w1, w2))
finder.apply_freq_filter(3)

Будем использовать likelihood ratio и mutual information. Ожидается, что likelihood выдаст хороший результат, а mi выдаст неплохой, лучше, чем у pmi (эталон безысходности)

In [11]:
finder.nbest(bigram_measures.likelihood_ratio, 10)

[('выдать', 'санкция'),
 ('удовлетворить', 'иск'),
 ('принять', 'решение'),
 ('удовлетворить', 'ходатайство'),
 ('наложить', 'арест'),
 ('признать', 'виновная'),
 ('санкционировать', 'арест'),
 ('отказать', 'удовлетворение'),
 ('освободить', 'залог'),
 ('признать', 'правота')]

In [16]:
finder.nbest(bigram_measures.mi_like, 10)

[('выдать', 'санкция'),
 ('наложить', 'арест'),
 ('освободить', 'залог'),
 ('удовлетворить', 'ходатайство'),
 ('удовлетворить', 'иск'),
 ('принять', 'решение'),
 ('санкционировать', 'арест'),
 ('прийти', 'вывод'),
 ('усилить', 'катастрофа'),
 ('пройти', 'прение')]

Мой топ-10 (на основании raw_freq и интуиции):

[('удовлетворить', 'иск'), ('удовлетворить', 'ходатайство'), ('принять', 'решение'), ('выдать', 'санкция'), ('признать', 'виновная'), ('наложить', 'арест'), ('санкционировать', 'арест'), ('отказать', 'удовлетворение'), ('вынести', 'решение'), ('признать', 'правота')]

In [18]:
gs = [('удовлетворить', 'иск'), ('удовлетворить', 'ходатайство'), ('принять', 'решение'), \
      ('выдать', 'санкция'), ('признать', 'виновная'), ('наложить', 'арест'), ('санкционировать', 'арест'), \
      ('отказать', 'удовлетворение'), ('вынести', 'решение'), ('признать', 'правота')]

In [27]:
print('%0.2f' % nltk.collocations.spearman_correlation(nltk.ranks_from_sequence(gs), \
                    nltk.ranks_from_sequence(finder.nbest(bigram_measures.likelihood_ratio, 10))))

0.87


In [28]:
print('%0.2f' % nltk.collocations.spearman_correlation(nltk.ranks_from_sequence(gs), \
                    nltk.ranks_from_sequence(finder.nbest(bigram_measures.mi_like, 10))))

-0.54


Получается, что (как и ожидалось) у likelihood_ratio корреляция со стандартом почти 1, а mutual information даёт более слабую корреляцию (да ещё и большáя часть в обратном порядке)

Кстати, проверим предположение про pmi:

In [29]:
print('%0.2f' % nltk.collocations.spearman_correlation(nltk.ranks_from_sequence(gs), \
                    nltk.ranks_from_sequence(finder.nbest(bigram_measures.pmi, 10))))

0.00


Доказано!

P.S. На основании взятого с потолка стандарта из 10 позиций, конечно, ничего утверждать нельзя, но на мой взгляд likelihood_ratio действительно даёт самый адекватный вариант